<a href="https://colab.research.google.com/github/ravi-prakash1907/Machine-Learning-for-Cyber-Security/blob/main/Classifiers/Naive%20Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Classifier 

**Steps:**  
1. Get the prediction labels' probability  
2. Get the probability for all the dependent varieables (labels') probabilities for corresponding possibilities  
3. Use Naive Bayes' Classifier  
4. Predict based on the Maximum A Posteriori (_MAP_)

In [4]:
## libraries
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('Datasets/stolenCars.csv')
df.head()

,Car-ID,Color,Type,Origin,Stolen?
0,C1,Red,Sports,Domestic,Yes
1,C2,Red,Sports,Domestic,No
2,C3,Red,Sports,Domestic,Yes
3,C4,Yellow,Sports,Domestic,No
4,C5,Yellow,Sports,Imported,Yes


In [6]:
df.shape

(10, 5)

In [7]:
df.describe()

,Car-ID,Color,Type,Origin,Stolen?
count,10,10,10,10,10
unique,10,2,2,2,2
top,C2,Red,Sports,Imported,No
freq,1,5,6,5,5


---

### Naive Bayes' Classifier
_It takes the tuples for set of the:_  
1. Dependent variables  
2. Values of any datapoint
3. Name of cloumn i.e. to be predicted

In [8]:
## gives rows with given set of cols
def getOneLabel(df,label,predLabel):
  tempDf = pd.read_csv('Datasets/stolenCars.csv')
  x = [df.loc[i][label] in predLabel for i in df.index]
  count = 0
  ind = []
  for val in x:
    if not val:
      ind.append(count)
    count += 1
  
  #print(ind)
  tempDf.drop(ind, inplace=True)
  return tempDf

In [9]:
## basic classifier
def naiveBayesianPredictor(df, labels, given, predCol, describe = False):
  possibleCases = list(df[predCol].unique())
  finalPredDescription = {}
  
  for possibleCase in possibleCases:
    tempDF = getOneLabel(df, predCol, [possibleCase])
    preds = len(tempDF) / len(df)

    for index in range(len(given)):
      partialProb = tempDF[labels[index]].value_counts() / len(tempDF)
      preds *= partialProb[given[index]]  #Naive bayesian classifier
    
    finalPredDescription[possibleCase] = preds
  
  ## converting predictions to pandas Series
  finalPredDescription = pd.Series(finalPredDescription)
  ## choosing MAP (Maximum A Posteriori)
  finalPred = str(finalPredDescription[finalPredDescription == max(finalPredDescription)].index[0])

  if describe:
    return finalPred,finalPredDescription
  else:
    return finalPred

In [10]:
## testing set
labels = ('Color','Type', 'Origin')
given = ('Red', 'SUV', 'Domestic')
predCol = 'Stolen?'

## prediction
finalPrediction = naiveBayesianPredictor(df, labels, given, predCol, describe=True)
probTable = pd.DataFrame([finalPrediction[1].index, finalPrediction[1].values], columns = ['Predicted Labels', 'Probability'])

## result
print("""Hence, as per final prediction: \nWill a car with specification {} be {}? : {} \n
Probability Table: """.format(given, 
             predCol,
             finalPrediction[0]))
pd.DataFrame([finalPrediction[1]])

Hence, as per final prediction: 
Will a car with specification ('Red', 'SUV', 'Domestic') be Stolen?? : No 

Probability Table: 


,Yes,No
0,0.024,0.072


--- 

## Intermediate Probability Tables

In [72]:
def getProbTable(df,dependentCol,predCol = 'Stolen?'):
  possibleCases = list(df[predCol].unique())
  possibleCases  # ['Yes', 'No']
  sets = pd.DataFrame()

  for possibleCase in possibleCases:
    #possibleCase = 'Yes'
    targetDF = getOneLabel(df, predCol, [possibleCase])
    PcolGIVENcase = targetDF[dependentCol].value_counts()/len(targetDF)

    header = "P("+dependentCol+"|"+possibleCase+")"
    sets[header] = PcolGIVENcase
    sets.append(PcolGIVENcase)
  return sets

In [74]:
## printing intermediate tables (for all three dependent columns)
for col in df.columns[1:-1]:
  print("\nProbability Table for",col,"-\n")
  x = getProbTable(df,col)
  print(x, "\n\n"+"-"*35+"\n")


Probability Table for Color -

        P(Color|Yes)  P(Color|No)
Red              0.6          0.4
Yellow           0.4          0.6 

-----------------------------------


Probability Table for Type -

        P(Type|Yes)  P(Type|No)
Sports          0.8         0.4
SUV             0.2         0.6 

-----------------------------------


Probability Table for Origin -

          P(Origin|Yes)  P(Origin|No)
Imported            0.6           0.4
Domestic            0.4           0.6 

-----------------------------------

